In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Поведенческие паттерны  
  
  
### Определяют алгоритмы и способы реализации взаимодействия различных объектов и классов.

* __Chain of Responsibility__
* Command
* __Iterator__
* Mediator
* Memento
* Observer
* State
* __Strategy__
* __Template Method__
* Visitor

---

## Iterator

### Задача  
  
Вам необходимо итерироваться по контейнеру. Порядок обхода для одной и той же коллекции может быть разным (прямой/обратный, по расположению в памяти, по линейному порядку, по заранее определенному алгоритму).

### Решение  
  
Вынесем поведение обхода коллекции из самой коллекции в отдельный класс.  

Объект-итератор будет отслеживать состояние обхода, текущую позицию в коллекции и сколько элементов ещё осталось обойти. Одну и ту же коллекцию смогут одновременно обходить различные итераторы, а сама коллекция не будет даже знать об этом.  
  
К тому же, если вам понадобится добавить новый способ обхода, вы сможете создать отдельный класс итератора, не изменяя существующий код коллекции.

![iterator](uml/iterator.png)

### Плюсы

 * упрощает классы хранения данных
 * позволяет реализовать различные способы обхода структуры данных
 * позволяет одновременно перемещаться по структуре данных в разные стороны

### Минусы

 * не оправдан, если можно обойтись простым циклом (если есть обращение по индексу за О(1)!)

---

## Chain of Responsibility

### Задачи (проблемы). Примеры  

 1. Вы реализуете систему авторизации на сайте. Есть несколько уровней доступа: "guest", "authorized guest", "editor", "admin". Необходимо последовательно предоставлять возможность “повысить” уровень доступа или заканчивать проверку, когда посетитель достиг своего уровня. Получается пайплайн захода на сайт является цепочкой _верификаций_.
 2. Пайплайн сборки вашего кода: запуск статического анализатора, разного рода хуки и линтеры, запуск тестов, непосредственная сборка. Также здесь может быть встроена нотификация об (не)успешной сборке, выгрузка артефактов (архив с бинарником программы) в общее хранилище, накат дистрибутива на тестовые машины с запуском системных тестов и тд и тп.
 3. Преобразование видео: предобработки, наложение фильтров, параллельная конвертация в MP4, AVI, MOV.
 
Все эти задачи представляют собой некое квазилинейное поведение с поочередными действиями.  
  
 * Надо обработать несколько запросов - но неизвестны обработчики заранее (можем динамически их выполнять).
 * Важно задать порядок обработки.
 * Набор объектов для обработки должен задаваться динамически.

### Решение  
  
Давайте превратим отдельные этапы в объекты классов. В нашем случае каждая проверка переедет в отдельный класс с единственным методом выполнения. Данные запроса, над которым происходит проверка, будут передаваться в метод как аргументы.  

Давайте свяжем объекты обработчиков в одну цепь. Каждый будет иметь указатель/ссылку на следующий обработчик в цепи. Таким образом, во время обработки можем по необходимости передать обработку следующему объекту в цепочке.  
  
Обработчик не обязательно должен передавать запрос дальше, причём эта особенность может быть использована по-разному.

![chain-of-responsibility](uml/chain-of-responsibility.png)

__Вопрос__: на какой другой паттерн по своей схеме похож Chain of Responsibility?

Не имеет базового объекта и может прервать выполнение на любом шаге.  
Допускается нелинейность, в отдельных случаях саморекурсивная обработка отдельного подмножества шагов.

In [2]:
from abc import ABC
from abc import abstractmethod
from typing import Any
from typing import Optional


class Handler(ABC):

    @abstractmethod
    def set_next(self, handler: "Handler") -> "Handler":
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass


class AbstractHandler(Handler):

    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)
        return None


class GuestHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "guest":
            return f"Hello, Anonymous Guest!"
        else:
            return super().handle(request)


class AuthorizedGuestHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        login = input("Input login: ")
        password = input("Input password: ")
        if login == "user" and password == "qwerty":  # dummy security
            if request == "authorized guest":
                return f"Hello, {login}!"
            else:
                return super().handle(request)
        else:
            return "Authorization failed!"


class EditorHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        secret_phrase = input("Input secret phrase: ")
        if secret_phrase == "secret":
            if request == "editor":
                return f"Hello, Editor!"
            else:
                return super().handle(request)
        else:
            return "Secret phrase is incorrect!"

        
class AdminHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        sms_token = input("Input sms token: ")
        if sms_token == "1111":
            if request == "admin":
                return f"Hello, Admin!"
            else:
                return super().handle(request)
        else:
            return "Token is incorrect!"


In [3]:
guest_handler = GuestHandler()
authorized_guest_handler = AuthorizedGuestHandler()
editor_handler = EditorHandler()
admin_handler = AdminHandler()

guest_handler \
    .set_next(authorized_guest_handler) \
    .set_next(editor_handler) \
    .set_next(admin_handler)

In [4]:
# try guest

guest_handler.handle("guest")

'Hello, Anonymous Guest!'

In [5]:
# try authorized guest

guest_handler.handle("authorized guest")

Input login: user
Input password: qwerty


'Hello, user!'

In [6]:
# try editor

guest_handler.handle("editor")

Input login: user
Input password: qwerty
Input secret phrase: nonsecret


'Secret phrase is incorrect!'

In [7]:
# try admin

guest_handler.handle("admin")

Input login: user
Input password: qwerty
Input secret phrase: secret
Input sms token: 1111


'Hello, Admin!'

### Плюсы

 * уменьшение зависимости между клиентом и обработчиком
 * каждая команда - отдельная ответственность
 * для того, чтобы дополнить поведение команды, можно применить наследование, а не изменять текущий код
 
Итого: реализует __Single Responsibility Principle__ и __Open-Closed Principle__.

### Минусы

 * какой-то запрос может остаться невыполненным (какие-то обработчики будут unreachable и тд)

---

## Strategy

### Задачи (проблемы). Примеры  
  1. Вы реализуете средство визуальной генерации нейросетей. Пользователь загружает собственный датасет, настраивает программу и запускает её на вашем сервере. При этом все шаги выполнения одинаковые, кроме, непосредственно, алгоритма обучения, сгенерированного пользователем.
  2. Прокладывание маршрутов в GPS между точкой А и точкой Б с помощию различных алгоритмов: Дейкстры, Форда-Фалкерсона, А*, Ida-Star и тд и тп.

**Случаи применения**
 * Надо использовать разные вариации одного алгоритма.
 * Есть множество похожих классов, отличающихся некоторым поведением.
 * Не хотим сильно раскрывать реализации алгоритмов.
 * Ветка - вариация алгоритма: чтобы не писать гору if/else или switch-ей.

### Решение  

Необходимо выделить определенные алгоритмы в отдельные классы, которые можно будет подменять для разных экземпляров
классов.

![strategy](uml/strategy.png)

In [8]:
from abc import ABC
from abc import abstractmethod
import sys


class Strategy(ABC):
    @abstractmethod
    def execute(self):
        pass


class Context:
    """
    Контекст определяет интерфейс, представляющий интерес для клиентов.
    """
    def __init__(self, strategy: Strategy):
        self._strategy = strategy

    @property
    def strategy(self) -> Strategy:
        return self._strategy

    @strategy.setter
    def strategy(self, strategy: Strategy):
        self._strategy = strategy

    def do_something(self):
        print("Before Strategy call")
        self.strategy.execute()
        print("After Strategy call", file=sys.stderr)
        
        
class HelloWorldStrategy(Strategy):
    def execute(self):
        print("Hello, World!", file=sys.stderr)
        
        
class Hello021Strategy(Strategy):
    def execute(self):
        print("Hello, Б05-021!", file=sys.stderr)
        
        
class Hello022Strategy(Strategy):
    def execute(self):
        print("Hello, Б05-022!", file=sys.stderr)
        
        
class Hello023Strategy(Strategy):
    def execute(self):
        print("Hello, Б05-023!", file=sys.stderr)

In [9]:
context = Context(HelloWorldStrategy())
context.do_something()

Before Strategy call


Hello, World!
After Strategy call


In [10]:
context.strategy = Hello021Strategy()
context.do_something()

Before Strategy call


Hello, Б05-021!
After Strategy call


In [11]:
context.strategy = Hello022Strategy()
context.do_something()

Before Strategy call


Hello, Б05-022!
After Strategy call


In [12]:
context.strategy = Hello023Strategy()
context.do_something()

Before Strategy call


Hello, Б05-023!
After Strategy call


### Плюсы

 * подмена алгоритмов на лету
 * изоляция код и данных алгоритмов от “интерфейсной” части
 * переход к делегированию ответственности
 * надо написать новую часть алгоритма - можно отнаследоваться (аналогично Chain of Responsibility)

### Минусы

 * много классов (ну куда же без этого)
 * необходимо разбираться во всех этих классах: кто что делает (обычно правильные именования позволяют этого избегать)

---

## Template Method

### Задачи (проблемы). Примеры  
 1. Часть вашего алгоритма содержит платформо-зависимый код(например, код создания файла в системе и небольшого свопа в него). Таких шагов алгоритма несколько и внесение директив на каждом крайне увеличивает объём кода.  
 2. Вы пишете программу для дата-майнинга в офисных документах в разных форматах: PDF, DOC, CSV. Программа должна извлекать из них полезную информацию. В какой-то момент вы заметили, что код всех трёх классов обработки документов хоть и отличается в части работы с файлами, но содержат довольно много общего в части самого извлечения данных. Было бы здорово избавится от повторной реализации алгоритма извлечения данных в каждом из классов.
 3. Разрабатываете игру с персонажами различных классов. Необходимо задать поведение этих героев, например, поведение при езде верхом или обращение с лассо или стрельбу из пулемета. Одинаково ли они должны все это делать?
  
Говоря человеческим языком, нам сперва необходимо задать скелет рабочей программы, а после его обращивать мясом.    
  
Более профессионально: необходимо организовать предоставление доступа к добавлению своей функциональности к некоторой части общего алгоритма путем переопределения некоторой зоны ответственности.

### Решение  
Давайте разобъем алгоритм на последовательность шагов, опишем эти шаги в отдельных методах и будем вызывать их в одном шаблонном методе друг за другом.  
  
Это позволит подклассам переопределять некоторые шаги алгоритма, оставляя без изменений его структуру и остальные шаги, которые для этого подкласса не так важны.

![template-method](uml/template-method.png)

In [13]:
from abc import ABC
from abc import abstractmethod
import sys


class AbstractMiner(ABC):

    def template_run_file_mining(self, filename):
        if self.is_accesable(filename):
            self.notify_started_processing(filename)
            self.process(filename)
            self.notify_finished_processing(filename)
        else:
            self.notify_inaccesable_file(filename)
            
    def notify_started_processing(self, filename):
        print(f"File {filename} is being processing", file=sys.stdout)
        
    def notify_finished_processing(self, filename):
        print(f"File {filename} has being processed", file=sys.stdout)
        
    def notify_inaccesable_file(self, filename):
        print(f"File {filename} is not accesable", file=sys.stderr)
            
    @abstractmethod
    def is_accesable(self, filename) -> None:
        pass

    @abstractmethod
    def process(self, filename) -> None:
        pass


class PdfMiner(AbstractMiner):

    def is_accesable(self, filename) -> None:
        return True

    def process(self, filename) -> None:
        print(f">> PDF file {filename} is being mining")
    
    
class DocMiner(AbstractMiner):
    
    def is_accesable(self, filename) -> None:
        return False

    def process(self, filename) -> None:
        print(f">> DOC file {filename} is being mining")
    

class CsvMiner(AbstractMiner):
    
    def is_accesable(self, filename) -> None:
        return True

    def process(self, filename) -> None:
        print(f">> CVS file {filename} is being mining")

In [14]:
PdfMiner().template_run_file_mining("file.pdf")

File file.pdf is being processing
>> PDF file file.pdf is being mining
File file.pdf has being processed


In [15]:
DocMiner().template_run_file_mining("file.doc")

File file.doc is not accesable


In [16]:
CsvMiner().template_run_file_mining("file.csv")

File file.csv is being processing
>> CVS file file.csv is being mining
File file.csv has being processed


### Плюсы

 * решает DRY (Don't Repeate Youself), т.е. облегчает повторное использование кода

### Минусы

 * жёстко ограничены скелетом существующего алгоритма
 * с ростом количества шагов шаблонный метод становится слишком сложно поддерживать

Проведите сравнение Стратегии и Шаблонного Метода.